# Semantic Search With Vector Store

In [78]:
import re

import spacy
from spacy.lang.ru.stop_words import STOP_WORDS
from razdel import sentenize

from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient

from tqdm.notebook import tqdm

In [31]:
spacy_model = spacy.load('ru_core_news_md')

### Data Preprocessing

В качестве исходного текста будем использовать роман-антиутопию Джорджа Оруэлла "1984": 

In [3]:
with open('1984.txt', 'r', encoding='utf-8') as f:
    src_text = f.read()

src_text[:100]

'Был холодный ясный апрельский день, и часы пробили тринадцать. Уткнув подбородок в грудь, чтобы спас'

Разобьем текст на сегменты. Будем использовать **small-to-big** подход, при этом короткие предложения (< 5 значимых токенов) будем объединять со следующими за ними. 

Это позволит частично улучшить сегментацию диалогов, поддерживая определенный "минимум" информации в сегменте и при этом не раздувая его размер (что потенциально улучшит последующую векторизацию).

In [44]:
CONTEXT_WINDOW_SIZE = 2 # sentence window radius (sentence +- context_radius)
MIN_USEFUL_TOKENS = 5 # minimum amount of non-stopword tokens in each segment
HYPERLINK_PATTERN = re.compile(r'\([0-9]+\)')

In [57]:
def preprocess_sentence(sentence):
    """Strips the sentence and removes (i)-type links and newlines from it."""
    sentence = sentence.text.strip().replace('\n\n', ' ').replace('\n', ' ')
    
    start = 0
    clean_sentence = []
    for match in HYPERLINK_PATTERN.finditer(sentence):
        end = match.start()
    
        clean_sentence.append(sentence[start:end])
    
        start = match.end()
    
    clean_sentence.append(sentence[start:])
    return ''.join(clean_sentence)

In [41]:
def count_useful_tokens(sentence):
    """Counts non-stopwords and non-punctuation tokens in the sentence."""
    sentence_pipe = spacy_model.pipe([sentence])
    
    return len([[w for w in doc if not w.is_stop and not w.is_punct] for doc in sentence_pipe][0])

In [58]:
sentences = sentenize(src_text)

segments = []
segment = []
token_count = 0
for sentence in tqdm(sentences, desc='Splitting the text into segments'):
    # clean each sentence
    clean_sentence = preprocess_sentence(sentence)

    # update useful tokens counter
    token_count += count_useful_tokens(clean_sentence)

    # add sentence to the segment
    segment.append(clean_sentence)

    # construct a segment
    if token_count >= MIN_USEFUL_TOKENS:
        segments.append(' '.join(segment))
        segment = []
        token_count = 0

len(segments)

Splitting the text into segments: 0it [00:00, ?it/s]

4365

In [60]:
segments[0], segments[1026], segments[-1]

('Был холодный ясный апрельский день, и часы пробили тринадцать.',
 'Он задумался, как задумывался уже не раз, а не сумасшедший ли он сам. Может быть, сумасшедший тот, кто в меньшинстве, в единственном числе.',
 'Окончательный переход на новояз был отложен до 2050 года именно с той целью, чтобы оставить время для предварительных работ по переводу.')

Добавим к каждому сегменту префикс:

In [61]:
prefixes = ('query: ', 'passage: ')
prefixed_segments = []

for segment in segments:
    prefixed_segments.append(prefixes[1] + segment)

prefixed_segments[0], prefixed_segments[1026], prefixed_segments[-1]

('passage: Был холодный ясный апрельский день, и часы пробили тринадцать.',
 'passage: Он задумался, как задумывался уже не раз, а не сумасшедший ли он сам. Может быть, сумасшедший тот, кто в меньшинстве, в единственном числе.',
 'passage: Окончательный переход на новояз был отложен до 2050 года именно с той целью, чтобы оставить время для предварительных работ по переводу.')

Преобразуем сегменты в langchain-документы:

In [66]:
documents = [Document(page_content=elem, metadata={'id': idx}) for idx, elem in enumerate(prefixed_segments)]
documents[0], documents[1026], documents[-1]

(Document(page_content='passage: Был холодный ясный апрельский день, и часы пробили тринадцать.', metadata={'id': 0}),
 Document(page_content='passage: Он задумался, как задумывался уже не раз, а не сумасшедший ли он сам. Может быть, сумасшедший тот, кто в меньшинстве, в единственном числе.', metadata={'id': 1026}),
 Document(page_content='passage: Окончательный переход на новояз был отложен до 2050 года именно с той целью, чтобы оставить время для предварительных работ по переводу.', metadata={'id': 4364}))

### Vector Store Launch

Векторизуем сегменты и поднимем векторную БД:

In [63]:
checkpoint = 'intfloat/multilingual-e5-large'

In [64]:
embeddings = HuggingFaceEmbeddings(
    model_name=checkpoint,
    model_kwargs={'device': 'cpu'},
)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Предварительно запустим docker-контейнер следующей командой:
```
docker run -p 6333:6333 qdrant/qdrant
```

In [70]:
url = 'http://localhost:6333'
qdrant_client = Qdrant.from_documents(
    documents,
    embeddings,
    url=url,
    # prefer_grpc=True,
    collection_name='qdrant_novels_1984',
)

### Search Through Vector Store

Воспользуемся интерфейсом langchain для поиска по созданной БД:

In [72]:
retriever = qdrant_client.as_retriever(search_type='mmr', search_kwargs={'k': 5})
retriever

VectorStoreRetriever(tags=['Qdrant', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x00000196A1FB7390>, search_type='mmr', search_kwargs={'k': 5})

In [76]:
def retrieve_segment_context(idx, segments):
    """Encapsulates the segment in its context window."""
    context = []
    left_border = max(0, idx - CONTEXT_WINDOW_SIZE)
    right_border = min(len(segments) - 1, idx + CONTEXT_WINDOW_SIZE) 
    
    for i in range(left_border, right_border + 1):
        context.append(segments[i])

    return ' '.join(context)

Приведем несколько примеров запросов:

In [75]:
queries = [
    'Что такое ангсоц?',
    'Кто является главным злодеем произведения?',
    'Перечисли основные министерства Англии.',
    'Расскажи о Джулии.'
]

In [77]:
for query in queries:
    print(f'Query: {query}')
    print('=' * 90)
    print(f'Best candidates:')
    
    best_segments = retriever.get_relevant_documents(prefixes[0] + query)
    
    for i, elem in enumerate(best_segments):
        contexted_elem = retrieve_segment_context(elem.metadata['id'], segments)
        print(f'{i+1}. {contexted_elem}')
        
    print()

Query: Что такое ангсоц?
Best candidates:
1. Теперь их падало на Лондон по двадцать-тридцать штук в неделю. Внизу на улице ветер трепал рваный плакат, на нем мелькало слово АНГСОЦ. Ангсоц. Священные устои ангсоца. Новояз, двоемыслие, зыбкость прошлого. У него возникло такое чувство, как будто он бредет по лесу на океанском дне, заблудился в мире чудищ и сам он — чудище. Он был один. Прошлое умерло, будущее нельзя вообразить. Есть ли какая-нибудь уверенность, что хоть один человек из живых — на его стороне?
2. Считалось, что, если класс капиталистов лишить собственности, наступит социализм; и капиталистов, несомненно, лишили собственности. У них отняли все — заводы, шахты, землю, дома, транспорт; а раз все это перестало быть частной собственностью, значит, стало общественной собственностью. Ангсоц, выросший из старого социалистического движения и унаследовавший его фразеологию, в самом деле выполнил главный пункт социалистической программы — с результатом, который он предвидел и к котор

### Conclusion

В предыдущую модель поиска было введено 2 улучшения:
1. **small-to-big сегментация**. Раньше каждый сегмент создавался путем соединения 5 последовательно идущих предложений. Это вызывало проблему вариативности размеров сегментов (особенно было заметно в диалогах) + длинные сегменты хуже векторизовались. Теперь же в качестве сегментов рассматриваются последовательности текстов, содержащие 5+ значимых (т.е. не являющихся ни стоп-словами, ни пунктуацией) токенов - обычно каждый сегмент это 1-2 предложения. После нахождения семантически близких запросу сегментов последние расширялись окружающим контекстом радиуса 2. Таким образом, векторизовались небольшие фрагменты текста (что улучшило качество поиска), а информации в каждом сегменте, в среднем, стало больше за счет расширения контекстом;
2. **применение mmr**. MMR позволил выдавать более разнообразные по содержанию выжимки, что помогло охватить больше релевантной информации по запросу.

**Итого**: Текущая модель работает заметно лучше своей предыдущей версии + размещена в полноценной векторной БД Qdrant (в локальном docker-контейнере) со всеми вытекающими из этого преимуществами.